<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/chmntz_Lab2/Lab2/chmntz_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative methods**
Carl **Chemntiz**

# **Abstract**
It this lab different types of iterative methods was implemented and discussed. Stationary iterative methods such as Jacobi iteration and Gauss-Seidel iteration was implemented to solve diagonally dominant system matrices and their benefits over direct methods was discussed. A Krylov subspace method, GMRES, that is able to solve a general square matrix was also implemented and its benefits over stationary iterative methods were presented. Lastly, the lab also contains a method that can solve nonlinear equations, scalar functions, Newton's method.

# **Set up environment**
To have access to the necessary modules you have to run this cell.

In [1]:
from google.colab import files

import numpy as np
from numpy.linalg import norm, inv
from scipy.optimize import fsolve

# Iterative methods requires an arbitrary tolerance
TOL = 1e-12

# **Introduction**

For massive systems of linear equations direct methods and matrix factorization will come at a huge computational cost, especially for dense matrices. For such systems, iterative methods are more memory efficient and at a lower computational cost by iteratively converging on an approximate solution $x$.

There are two types of iterative methods, **stationary iterative methods** and **Krylov subspace methods**.  

## Stationary iterative methods
Stationary iterative methods are formulated as a linear fixed point iteration of the form
$$x^{(k+1)}=Mx^{(k)}+c,$$
where $M\in\mathbb{R}^{n\times n}$ is the iteration matrix and $c\in\mathbb{R}^n$ is the constant vector. The converge criterion is that $\|M\|< 1$.

The most simple stationary iterative method is Richardson iteration with $M_R=I-\alpha A$ and $c=\alpha b$,
$$x^{(k+1)}=(I-\alpha A)x^{(k)}+\alpha b,$$
where $\alpha$ is the step length. Convergance can be improved by precondition the system by multiplying both sides by an approximate inverse of $A$, $B\approx A^{-1}$ from the left,
$$BAx=Bb,$$
where $x$ still is the same solution. This precondition is left preconditioning, which for Richardson iteration gives left preconditioned Richardson iteration,
$$x^{(k+1)}=(I-\alpha BA)x^{(k)}+\alpha Bb.$$
There also exists right preconditioning, however, it is not relevant to this lab.

Matrix splitting an technique to formulate an system of linear equations, $Ax=b$, as a stationary iterative methods but splitting the system matrix into a sum,
$$A=A_1+A_2,$$
where $A_1$ is a nonsingulat matrix which is easy to invert.

## Krylov subspace methods
Krylov subspace method approximates an solution to the system of linear equations $Ax=b$,
$x^{(k)}\approx x$
in the Krylov subspace of $\mathbb{R}^n$.
The Krylov subspace is the subspace consturcted by $A$ and $b$,
$$\kappa_k=\text{span}(b,Ab,\dots,A^{k-1}b).$$

## Newtons method
There also exists iterative methods to approximate solutions (roots) to nonlinear equations. One such method is Newton's method which converges on a solution from an initial guess. However, it requires that the residual is differentiable.

# **Methods**

## Jacobi iteration for $Ax=b$
Jacobi iteration is an algorithm to approximate solution $x$ to $Ax=b$ where $A$ is a strictly diagonally dominant matrix.
$$|a_{ii}|>\sum_{j\neq u}|a_{ij}|.$$
The method is based on matrix splitting using a diagonal matrix,
$$A_1=D,\hspace{1em}A_2=A-D,$$
where $D=\text{diag}(A)$.
This gives the iteration matrix $M_J$
$$M_J=-L^{-1}(A-L)=I-D^{-1}Am$$
and the constant vector $c$
$$c=D^{-1}b.$$
Since matrix splitting is a stationary iterative method, we get that the linear iteration

$$x^{(k+1)}=M_Jx^{(k)}+c=(I-D^{-1}A)x^{(k)}+D^{-1}b.$$
Jacobi iteration is equivalent to left Jacobi preconditioned Richardson iteration with $B=D^{-1}$ and $\alpha=1$.

From chapter 7, example 7.8 presents the Jacobi iteration in terms of the components of $A=(a_{ij})$,

$$x_i^{(k+1)}=a_{ii}^{-1}\Big(b_i-\sum_{j\neq i}a_{ij}x_j^{(k)}\Big),\hspace{1em}\forall i$$

In [2]:
def jacobi_iteration(A: np.array, b: np.array) -> np.array:
    n = len(b)
    x = np.zeros(n)

    while norm(A @ x - b) > TOL:
        x_k = x.copy()
        for i in range(n):
            a_inv = 1 / A[i,i]
            row_sum = 0
            for j in range(n):
                if j == i: continue
                row_sum += A[i,j] * x_k[j]
            x[i] = a_inv * (b[i] - row_sum)
    return x

However, convergance of an iterative method can be improved by using *relaxation*. Relaxation is an method of setting the degree of diagonal dominance by a parameter $\omega>0$. This gives us the damped Jacobi iteration.
$$x^{(k+1)}=x^{(k)}+\omega D^{-1}(b-Ax^{(k)})$$
The book (chapter 7, section 7) also suggests an technique to improve convergance even further using *successive over-relaxation*, however, this technique was not implemented.

In [3]:
def damped_jacobi_iteration(A: np.array, b: np.array) -> np.array:
    x = b.copy()
    D = np.diag(np.diag(A))
    M = np.eye(len(b)) - inv(D) @ A
    # c = inv(D) @ b
    w = 0.7

    while norm(A @ x - b) > TOL:
        # x = M @ x + c
        x = x + w * inv(D) @ (b - A @ x)
    return x

## Gauss-Seidel iteration for $Ax=b$
Gauss-Seidel iteration is similar to Jacobi iteration, using the same principles but splitting the matrix using the lower triangular matrix,
$$A_1=L,\hspace{1em}A_2=A-L.$$
Unlike Jacobi iteration, $A$ does not have to be strictly diagonally dominant, and can be irreducibly diagonally dominant aswell.
Similarily the iteration matrix is
$$M_{GS}=-L^{-1}(A-L)=I-L^{-1}A,$$
and the vector becomes
$$c=L^{-1}b.$$
Gauss-Seidel iteration is equivalent to left preconditioned Richardson iteration with $B=L^{-1}$ and $\alpha=1$.

The form expressed in terms of components is

$$x_i^{(k+1)}=a_{ii}^{-1}\Big(b_i-\sum_{j<i}a_{ij}x_j^{(k+1)}-\sum_{j>i}a_{ij}x_j^{(k)}\Big),\hspace{1em}\forall i$$

In [4]:
def gauss_seidel_iteration(A: np.array, b: np.array) -> np.array:
    n = len(b)
    x = np.zeros(n)

    while norm(A @ x - b) > TOL:
        x_k = x.copy()
        for i in range(n):
            a_inv = 1 / A[i,i]
            row_sum = 0
            for j in range(i):
                row_sum += A[i,j] * x[j]
            for j in range(i+1, n):
                row_sum += A[i,j] * x_k[j]
            x[i] = a_inv * (b[i] - row_sum)
    return x

As Gauss-Seidel iteration is a stationary iterative method, the method can be damped using relaxation.

In [5]:
def damped_gauss_seidel_iteration(A: np.array, b: np.array) -> np.array:
    x = b.copy()
    L = np.tril(A)
    M = np.eye(len(b)) - inv(L) @ A
    c = inv(L) @ b
    w = 0.7

    while norm(A @ x - b) > TOL:
        # x = M @ x + c
        x = x + w * inv(L) @ (b - A @ x)
    return x

## Newton's method for scalar nonlinear equation $f(x)=0$
Newton's method is an iterative algorithm to approxiamte the roots of a function based on an initial guess. The function $f$ must be a differentiable function.
$$x^{(k+1)}=x^{(k)}-\frac{f(x^{(k)})}{f'(x^{(k)})}$$

Since Newton's method requires the derivative of a function, a method to approximate the finite difference was implemented. Central difference was chosen due to its higher accuracy compared to forward and backward differences.

In [6]:
def derivate(f, x: float) -> float:
    h = 1e-15
    return (f(x + h) - f(x - h)) / (2 * h)

In this lab `newtons_method` was implemented based on algorithm 8.2 from the book *Methods in Computational Science*, which itself was an implementation of the equation stated above.

In [7]:
def newtons_method(f, x: float) -> float:
    while abs(f(x)) > TOL:
        df = derivate(f, x)
        if df == 0: raise ArithmeticError
        x = x - (f(x) / df)
    return x

## GMRES method for $Ax=b$
Generalized minimal residual method (GMRES) approximates solution $x$ to the system of linear solution $Ax=b$ by minimizing the norm of the residual $r^{(k)}=b-Ax^{(k)}$.
This can be done using the least squares problem
$$\min_{x^{(k)}\in\kappa_k}\|b-Ax^{(k)}\|.$$
GMRES utilizes the Arnoldi algorithm which computes an orthonormal basis of the same space
and thus the approximation can be expressed as $x^{(k)}=Q_\kappa y$ with a vector $y\in\mathbb{R}^k$ with the coordinates of $x^{(k)}$ in the orthonormal basis. Thus,
$$\min_{y\in\mathbb{R}^k}\|b-AQ_\kappa y\|.$$
This makes the algorithm numerically stable. Had the approximation been expressed a linear combination of the Krylov vectors, the algorithm would have been numerically unstable.

Arnoldi iteration is an algorithm to approximate eigenvalues and eigenvectors of a $n\times n$ matrix $A$ by constructing an orthonormal basis $Q_k$,
$$AQ_k=Q_{k+1}H_k,$$
where $Q$ is the basis vectors of the Krylov subspace and $H$ is the upper Hessenberg matrix. It is based on modified Gram-Schmidt iteration with an initial vector $b$ which is used to get the first element of $Q$, $q_1=b/\|b\|$.

Arnoldi iteration was implemented based on algorithm 7.3 from the book *Methods in Computation Science*.

In [8]:
def arnoldi_iteration(A: np.array, b: np.array, k: int) -> np.array:
    Q = np.zeros((A.shape[0], k+1))
    H = np.zeros((k+1, k))
    Q[:,0] = b / norm(b)

    for j in range(1,k+1):
        v_j = A @ Q[:,j-1]
        for i in range(0,j):
            H[i,j-1] = Q[:,i] @ v_j
            v_j -= H[i,j-1] * Q[:,i]
        H[j, j-1] = norm(v_j)
        Q[:, j] = v_j / H[j, j-1]
    return Q, H

The GMRES algorithm itself was implemented based on algorithm 7.2 from the book *Methods in Computational Science*. In this implementation, Numpy's `linalg.lstsq` function was used to solve least squares problem.

In [9]:
def gmres(A: np.array, b: np.array) -> np.array:
    k_max = 1000
    Q = np.zeros((A.shape[0], k_max))
    Q[:,0] = b / norm(b)

    for k in range(1, k_max):
        e1 = np.zeros(k+1)
        e1[0] = 1

        Q, H = arnoldi_iteration(A, b, k)
        y = np.linalg.lstsq(H, norm(b)*e1, rcond=None)[0]
        r = (norm(b) * e1) - (H @ y)

        if norm(r) < TOL: break;
    x = Q[:, 0:k] @ y
    return x

# **Results**

## Jacobi iteration for $Ax=b$
A psuedorandom strictly diagonally dominant system matrix $A$ and psuedorandom vector $b$ of the same length is constructed.

In [10]:
A = np.random.random((5,5)) + 5 * np.eye(5)
b = np.random.rand(5)

The convergance of residual test $||Ax-b||$ is done by calculating $Ax$, where $x$ is the approximated solution and comparing it to vector $b$. If they are the same, i.e. $||Ax-b||=0$, then the algorithm converged on the true solution.

As two algorithm were implemented, the test was done on both.

In [11]:
print(f"||Ax-b||=0: {np.allclose(A @ jacobi_iteration(A,b), b)}")
print(f"||Ax-b||=0: {np.allclose(A @ damped_jacobi_iteration(A,b), b)}")

||Ax-b||=0: True
||Ax-b||=0: True


Using Numpy's own solver of system of linear equations to calculate the solution, it was compared to the user implemented solution. If they are the same, the user implemented solution converged on the same solution as the pre-built algorithm.

In [12]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), jacobi_iteration(A,b))}")
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), damped_jacobi_iteration(A,b))}")

||x-y||=0: True
||x-y||=0: True


## Gauss-Seidel iteration for $Ax=b$
A psuedorandom strictly diagonally dominant system matrix $A$ and psuedorandom vector $b$ of the same length is constructed.

In [13]:
A = np.random.random((5,5)) + 5 * np.eye(5)
b = np.random.rand(5)

The convergance of residual test $||Ax-b||$ is done by calculating $Ax$, where $x$ is the approximated solution and comparing it to vector $b$. If they are the same, i.e. $||Ax-b||=0$, then the algorithm converged on the true solution.

In [14]:
print(f"||Ax-b||=0: {np.allclose(A @ gauss_seidel_iteration(A,b), b)}")
print(f"||Ax-b||=0: {np.allclose(A @ damped_gauss_seidel_iteration(A,b), b)}")

||Ax-b||=0: True
||Ax-b||=0: True


Using Numpy's own solver of system of linear equations to calculate the solution, it was compared to the user implemented solution. If they are the same, the user implemented solution converged on the same solution as the pre-built algorithm.

In [15]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), gauss_seidel_iteration(A,b))}")
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), damped_gauss_seidel_iteration(A,b))}")

||x-y||=0: True
||x-y||=0: True


## Newton's method for scalar nonlinear equation $f(x)=0$
A nonlinear equation is declared as $f(x)$ and newton's method is used to approximate the root with an initial guess of $x_0=0$.

In [16]:
f = lambda x: 4 * x**3 + 2 * x**2 + 8 * x + 6
x = newtons_method(f, 0)

The approximated solution $x$ is inserted into the function $f$ and should yeild 0 if the algorithm was implemented correctly.

In [17]:
print(f"|f(x)|=0: {np.isclose(f(x),0)}")

|f(x)|=0: True


The approximated solution $x$ was compared to solution calculated by Scipy's `fsolve` function. If the algorithm was correctly implemented, they should be the same.

In [18]:
print(f"|x-y|=0: {np.isclose(x, fsolve(f, 0))[0]}")

|x-y|=0: True


## GMRES methods for $Ax=b$
A psuedorandom system of linear equations, $Ax=b$, was constructed with an unknown solution $x$. Since GMRES is an Krylov subspace method, it should be able to solve any general $n\times n$ matrix.

In [19]:
A = np.random.random((5,5))
b = np.random.rand(5)

The convergance of residual test $||Ax-b||$ is done by calculating $Ax$, where $x$ is the approximated solution and comparing it to vector $b$. If they are the same, i.e. $||Ax-b||=0$, then the algorithm converged on the true solution.

In [20]:
print(f"||Ax-b||=0: {np.allclose(A @ gmres(A,b), b)}")

||Ax-b||=0: True


Using Numpy's own solver of system of linear equations to calculate the solution, it was compared to the user implemented solution. If they are the same, the user implemented solution converged on the same solution as the pre-built algorithm.

In [21]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), gmres(A,b))}")

||x-y||=0: True


# **Discussion**
All results were as expected and all implmented algorithms passed their tests.
It was particular interesting to me that both Jacobi and Gauss-Seidel iteration can ee implemented using matrices and in terms of components. It should be noted that damped Jacobi and Gauss-Seidel iteration can be implemented in terms of components aswell, but due to time restrictions I only implemented the damped version using matrices. It would also have been interesting to measure which of the implementations (matrices vs component) was faster. In theory, the matrix implementation should be faster as linear algebra is very optimized on modern software.

While the stationary iterative methods were more simple to implement, GMRES was more useful, able to compute any general square matrix. I chose to use the already implemented least squares problem algorithm in this lab, however, I could also have used the implementation I did during the Matrix factorization lab. Since there was no restrictions of using already existing functions, there was no reason to use the user implemented solution.